In [85]:
!python module_preprocessing.py --default=True --train_ann='/input/train_annotations_equal.json' --dataset_type=1 --test_ann='/input/test_annotations_equal.json' --train_split=0.05

Creating new train/test split
Loaded annotataions file: /application/input/annotations.json
Dataset equal annotated/not annotated images.
Saved 100233 entries in /application/input/train_annotations_equal.json and 5275 in /application/input/test_annotations_equal.json


In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.version.cuda)

1.6.0 True 10.1


In [2]:
import os
import copy
import json
import pycocotools
import random 

import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

assert torch.__version__.startswith("1.6")

import detectron2
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg

from detectron2.data import MetadataCatalog,DatasetMapper,build_detection_train_loader,build_detection_test_loader
from detectron2.data import detection_utils as utils
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances 

from detectron2.evaluation import COCOEvaluator, inference_on_dataset

from detectron2.projects.deeplab import add_deeplab_config, build_lr_scheduler

In [3]:
PATH = os.path.abspath(os.getcwd())

register_coco_instances("my_dataset_train_v2",{},PATH + "/input/train_annotations_equal.json",PATH + "/input/train_v2/")
register_coco_instances("my_dataset_val_v2",{},PATH + "/input/test_annotations_equal.json",PATH + "/input/train_v2/")

my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train_v2")
dataset_dicts = DatasetCatalog.get("my_dataset_train_v2")

[11/30 15:34:59 d2.data.datasets.coco]: Loading /application/input/train_annotations_equal.json takes 1.10 seconds.
[11/30 15:35:00 d2.data.datasets.coco]: Loaded 100233 images in COCO format from /application/input/train_annotations_equal.json


In [4]:
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    # List of transforms https://detectron2.readthedocs.io/modules/data_transforms.html
    # Add saturation, add shear orsmth.
    transform_list = [
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
                      T.RandomLighting(0.05),
                      T.RandomRotation((-0.2,0.2))
                     ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

In [5]:
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)
    
    @classmethod
    def build_lr_scheduler(cls, cfg, optimizer):
        return build_lr_scheduler(cfg, optimizer)


In [6]:
#Call from anywhere else. 
#!tensorboard --logdir=run_equal --host=0.0.0.0
#http://0.0.0.0:6006/#scalars

TensorFlow installation not found - running with reduced feature set.
TensorBoard 2.4.0 at http://0.0.0.0:6006/ (Press CTRL+C to quit)
^C


In [8]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v2",) 
cfg.DATASETS.TEST = ("my_dataset_val_v2",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 20000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)
cfg.MAX_SIZE_TRAIN = 256 #Max image size 
cfg.OUTPUT_DIR = "./run_equal"
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.LR_SCHEDULER_NAME = "WarmupCosineLR" #avoid getting stuck in local minima. 

In [11]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #Truncated image -> https://github.com/keras-team/keras/issues/5475

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

[11/30 18:12:49 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/30 18:13:10 d2.data.datasets.coco]: Loading /application/input/train_annotations_equal.json takes 21.21 seconds.
[11/30 18:13:11 d2.data.datasets.coco]: Loaded 100233 images in COCO format from /application/input/train_annotations_equal.json
[11/30 18:13:16 d2.data.build]: Using training sampler TrainingSampler
[11/30 18:13:16 d2.data.common]: Serializing 100233 elements to byte tensors and concatenating them all ...
[11/30 18:13:17 d2.data.common]: Serialized dataset takes 27.94 MiB
[11/30 18:13:19 d2.engine.train_loop]: Starting training from iteration 10000
[11/30 18:13:27 d2.utils.events]:  eta: 1:07:50  iter: 10019  total_loss: 0.3736  loss_cls: 0.0298  loss_box_reg: 0.08521  loss_mask: 0.2025  loss_rpn_cls: 0.003358  loss_rpn_loc: 0.005304  time: 0.4173  data_time: 0.0270  lr: 0.001  max_mem: 3702M
[11/30 18:13:36 d2.utils.events]:  eta: 1:10:13  iter: 10039  total_loss: 0.4632  loss_cls: 0.02541  loss_box_reg: 0.07654  loss_mask: 0.2365  loss_rpn_cls: 0.004563  loss_rpn_loc:

[11/30 18:17:42 d2.utils.events]:  eta: 1:06:11  iter: 10619  total_loss: 0.3851  loss_cls: 0.02503  loss_box_reg: 0.08805  loss_mask: 0.2246  loss_rpn_cls: 0.003487  loss_rpn_loc: 0.005104  time: 0.4240  data_time: 0.0099  lr: 0.001  max_mem: 3702M
[11/30 18:17:51 d2.utils.events]:  eta: 1:06:02  iter: 10639  total_loss: 0.3852  loss_cls: 0.01977  loss_box_reg: 0.0776  loss_mask: 0.2659  loss_rpn_cls: 0.006352  loss_rpn_loc: 0.006164  time: 0.4240  data_time: 0.0100  lr: 0.001  max_mem: 3702M
[11/30 18:17:59 d2.utils.events]:  eta: 1:05:55  iter: 10659  total_loss: 0.496  loss_cls: 0.03034  loss_box_reg: 0.1014  loss_mask: 0.2713  loss_rpn_cls: 0.003457  loss_rpn_loc: 0.01414  time: 0.4241  data_time: 0.0100  lr: 0.001  max_mem: 3702M
[11/30 18:18:08 d2.utils.events]:  eta: 1:05:46  iter: 10679  total_loss: 0.3619  loss_cls: 0.03171  loss_box_reg: 0.09928  loss_mask: 0.1982  loss_rpn_cls: 0.003999  loss_rpn_loc: 0.004452  time: 0.4242  data_time: 0.0105  lr: 0.001  max_mem: 3702M
[11/

[11/30 18:22:19 d2.utils.events]:  eta: 1:02:10  iter: 11259  total_loss: 0.521  loss_cls: 0.05067  loss_box_reg: 0.1192  loss_mask: 0.2226  loss_rpn_cls: 0.006924  loss_rpn_loc: 0.01565  time: 0.4275  data_time: 0.0103  lr: 0.001  max_mem: 3702M
[11/30 18:22:27 d2.utils.events]:  eta: 1:02:05  iter: 11279  total_loss: 0.4486  loss_cls: 0.03803  loss_box_reg: 0.08788  loss_mask: 0.2524  loss_rpn_cls: 0.01124  loss_rpn_loc: 0.009638  time: 0.4276  data_time: 0.0101  lr: 0.001  max_mem: 3702M
[11/30 18:22:36 d2.utils.events]:  eta: 1:01:58  iter: 11299  total_loss: 0.4844  loss_cls: 0.04409  loss_box_reg: 0.1287  loss_mask: 0.2596  loss_rpn_cls: 0.01109  loss_rpn_loc: 0.007744  time: 0.4277  data_time: 0.0099  lr: 0.001  max_mem: 3702M
[11/30 18:22:45 d2.utils.events]:  eta: 1:01:52  iter: 11319  total_loss: 0.4556  loss_cls: 0.04414  loss_box_reg: 0.1304  loss_mask: 0.2571  loss_rpn_cls: 0.006446  loss_rpn_loc: 0.01321  time: 0.4278  data_time: 0.0099  lr: 0.001  max_mem: 3702M
[11/30 1

[11/30 18:26:55 d2.utils.events]:  eta: 0:58:09  iter: 11899  total_loss: 0.5269  loss_cls: 0.02921  loss_box_reg: 0.092  loss_mask: 0.2699  loss_rpn_cls: 0.003937  loss_rpn_loc: 0.006533  time: 0.4287  data_time: 0.0102  lr: 0.001  max_mem: 3702M
[11/30 18:27:04 d2.utils.events]:  eta: 0:58:00  iter: 11919  total_loss: 0.4333  loss_cls: 0.03955  loss_box_reg: 0.1277  loss_mask: 0.2254  loss_rpn_cls: 0.006222  loss_rpn_loc: 0.01723  time: 0.4287  data_time: 0.0102  lr: 0.001  max_mem: 3702M
[11/30 18:27:13 d2.utils.events]:  eta: 0:57:54  iter: 11939  total_loss: 0.3756  loss_cls: 0.03813  loss_box_reg: 0.09972  loss_mask: 0.2167  loss_rpn_cls: 0.007535  loss_rpn_loc: 0.01394  time: 0.4288  data_time: 0.0101  lr: 0.001  max_mem: 3702M
[11/30 18:27:21 d2.utils.events]:  eta: 0:57:45  iter: 11959  total_loss: 0.4401  loss_cls: 0.04233  loss_box_reg: 0.1333  loss_mask: 0.2072  loss_rpn_cls: 0.003252  loss_rpn_loc: 0.008409  time: 0.4289  data_time: 0.0107  lr: 0.001  max_mem: 3702M
[11/30

...

[11/30 19:21:18 d2.utils.events]:  eta: 0:09:28  iter: 18679  total_loss: 0.3908  loss_cls: 0.02782  loss_box_reg: 0.1018  loss_mask: 0.2396  loss_rpn_cls: 0.002634  loss_rpn_loc: 0.003309  time: 0.4310  data_time: 0.0096  lr: 0.001  max_mem: 3702M
[11/30 19:21:27 d2.utils.events]:  eta: 0:09:20  iter: 18699  total_loss: 0.3337  loss_cls: 0.03401  loss_box_reg: 0.09453  loss_mask: 0.1823  loss_rpn_cls: 0.001307  loss_rpn_loc: 0.004406  time: 0.4310  data_time: 0.0100  lr: 0.001  max_mem: 3702M
[11/30 19:21:36 d2.utils.events]:  eta: 0:09:11  iter: 18719  total_loss: 0.5186  loss_cls: 0.036  loss_box_reg: 0.134  loss_mask: 0.2212  loss_rpn_cls: 0.005392  loss_rpn_loc: 0.0152  time: 0.4310  data_time: 0.0099  lr: 0.001  max_mem: 3702M
[11/30 19:21:44 d2.utils.events]:  eta: 0:09:02  iter: 18739  total_loss: 0.4081  loss_cls: 0.03658  loss_box_reg: 0.08557  loss_mask: 0.2502  loss_rpn_cls: 0.004314  loss_rpn_loc: 0.01033  time: 0.4310  data_time: 0.0098  lr: 0.001  max_mem: 3702M
[11/30 1

[11/30 19:25:56 d2.utils.events]:  eta: 0:04:52  iter: 19319  total_loss: 0.4682  loss_cls: 0.02951  loss_box_reg: 0.06671  loss_mask: 0.2628  loss_rpn_cls: 0.003442  loss_rpn_loc: 0.008477  time: 0.4311  data_time: 0.0097  lr: 0.001  max_mem: 3702M
[11/30 19:26:04 d2.utils.events]:  eta: 0:04:44  iter: 19339  total_loss: 0.4579  loss_cls: 0.04111  loss_box_reg: 0.1049  loss_mask: 0.2343  loss_rpn_cls: 0.004852  loss_rpn_loc: 0.01371  time: 0.4311  data_time: 0.0099  lr: 0.001  max_mem: 3702M
[11/30 19:26:13 d2.utils.events]:  eta: 0:04:35  iter: 19359  total_loss: 0.394  loss_cls: 0.02686  loss_box_reg: 0.08708  loss_mask: 0.2308  loss_rpn_cls: 0.002414  loss_rpn_loc: 0.008853  time: 0.4311  data_time: 0.0098  lr: 0.001  max_mem: 3702M
[11/30 19:26:22 d2.utils.events]:  eta: 0:04:27  iter: 19379  total_loss: 0.4499  loss_cls: 0.02924  loss_box_reg: 0.09437  loss_mask: 0.1938  loss_rpn_cls: 0.003533  loss_rpn_loc: 0.006164  time: 0.4311  data_time: 0.0102  lr: 0.001  max_mem: 3702M
[11

[11/30 19:30:33 d2.utils.events]:  eta: 0:00:17  iter: 19959  total_loss: 0.3436  loss_cls: 0.01795  loss_box_reg: 0.08037  loss_mask: 0.1962  loss_rpn_cls: 0.004236  loss_rpn_loc: 0.004077  time: 0.4312  data_time: 0.0096  lr: 0.001  max_mem: 3702M
[11/30 19:30:42 d2.utils.events]:  eta: 0:00:08  iter: 19979  total_loss: 0.4854  loss_cls: 0.05371  loss_box_reg: 0.1229  loss_mask: 0.221  loss_rpn_cls: 0.008035  loss_rpn_loc: 0.0135  time: 0.4312  data_time: 0.0103  lr: 0.001  max_mem: 3702M
[11/30 19:30:57 d2.data.datasets.coco]: Loaded 5275 images in COCO format from /application/input/test_annotations_equal.json
[11/30 19:30:57 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/30 19:30:57 d2.data.common]: Serializing 5275 elements to byte tensors and concatenating them all ...
[11/30 19:30:57 d2.data.common]: Serialized dataset takes 1.44 MiB
WARNING [11/30 19:30:57 d2

[11/30 19:36:10 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 0.65 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 0.07 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.741
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.451
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.290
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.672
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.682
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.287
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | m

[11/30 19:38:50 d2.evaluation.evaluator]: Inference done 2665/5275. 0.0527 s / img. ETA=0:02:33
[11/30 19:38:55 d2.evaluation.evaluator]: Inference done 2753/5275. 0.0527 s / img. ETA=0:02:27
[11/30 19:39:00 d2.evaluation.evaluator]: Inference done 2839/5275. 0.0527 s / img. ETA=0:02:22
[11/30 19:39:05 d2.evaluation.evaluator]: Inference done 2926/5275. 0.0527 s / img. ETA=0:02:17
[11/30 19:39:10 d2.evaluation.evaluator]: Inference done 3009/5275. 0.0527 s / img. ETA=0:02:12
[11/30 19:39:15 d2.evaluation.evaluator]: Inference done 3096/5275. 0.0527 s / img. ETA=0:02:07
[11/30 19:39:20 d2.evaluation.evaluator]: Inference done 3182/5275. 0.0527 s / img. ETA=0:02:02
[11/30 19:39:25 d2.evaluation.evaluator]: Inference done 3268/5275. 0.0527 s / img. ETA=0:01:57
[11/30 19:39:30 d2.evaluation.evaluator]: Inference done 3352/5275. 0.0527 s / img. ETA=0:01:52
[11/30 19:39:35 d2.evaluation.evaluator]: Inference done 3436/5275. 0.0527 s / img. ETA=0:01:47
[11/30 19:39:40 d2.evaluation.evaluator]

In [12]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v2",) 
cfg.DATASETS.TEST = ("my_dataset_val_v2",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00033  
cfg.SOLVER.MAX_ITER = 25000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)
cfg.MAX_SIZE_TRAIN = 512 #Max image size 
cfg.LR_SCHEDULER_NAME = "WarmupCosineLR" #avoid getting stuck in local minima. 
cfg.OUTPUT_DIR = "./run_equal"
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

In [13]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

[11/30 19:42:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/30 19:42:53 d2.data.datasets.coco]: Loading /application/input/train_annotations_equal.json takes 1.41 seconds.
[11/30 19:42:54 d2.data.datasets.coco]: Loaded 100233 images in COCO format from /application/input/train_annotations_equal.json
[11/30 19:43:00 d2.data.build]: Using training sampler TrainingSampler
[11/30 19:43:00 d2.data.common]: Serializing 100233 elements to byte tensors and concatenating them all ...
[11/30 19:43:00 d2.data.common]: Serialized dataset takes 27.94 MiB
[11/30 19:43:01 d2.engine.train_loop]: Starting training from iteration 20000
[11/30 19:43:10 d2.utils.events]:  eta: 0:36:07  iter: 20019  total_loss: 0.2631  loss_cls: 0.01977  loss_box_reg: 0.03585  loss_mask: 0.177  loss_rpn_cls: 0.003592  loss_rpn_loc: 0.00514  time: 0.4429  data_time: 0.0270  lr: 0.001  max_mem: 3702M
[11/30 19:43:19 d2.utils.events]:  eta: 0:35:09  iter: 20039  total_loss: 0.4894  loss_cls: 0.0178  loss_box_reg: 0.04543  loss_mask: 0.311  loss_rpn_cls: 0.003825  loss_rpn_loc: 0.0

[11/30 19:47:34 d2.utils.events]:  eta: 0:31:46  iter: 20619  total_loss: 0.4072  loss_cls: 0.01543  loss_box_reg: 0.03484  loss_mask: 0.274  loss_rpn_cls: 0.0046  loss_rpn_loc: 0.006884  time: 0.4393  data_time: 0.0098  lr: 0.001  max_mem: 3702M
[11/30 19:47:43 d2.utils.events]:  eta: 0:31:38  iter: 20639  total_loss: 0.4027  loss_cls: 0.03041  loss_box_reg: 0.06499  loss_mask: 0.2587  loss_rpn_cls: 0.003993  loss_rpn_loc: 0.01058  time: 0.4395  data_time: 0.0102  lr: 0.001  max_mem: 3702M
[11/30 19:47:52 d2.utils.events]:  eta: 0:31:31  iter: 20659  total_loss: 0.377  loss_cls: 0.02968  loss_box_reg: 0.06328  loss_mask: 0.2697  loss_rpn_cls: 0.005196  loss_rpn_loc: 0.005447  time: 0.4399  data_time: 0.0105  lr: 0.001  max_mem: 3702M
[11/30 19:48:01 d2.utils.events]:  eta: 0:31:24  iter: 20679  total_loss: 0.376  loss_cls: 0.0292  loss_box_reg: 0.07166  loss_mask: 0.2193  loss_rpn_cls: 0.0049  loss_rpn_loc: 0.01044  time: 0.4402  data_time: 0.0103  lr: 0.001  max_mem: 3702M
[11/30 19:

[11/30 19:52:20 d2.utils.events]:  eta: 0:27:37  iter: 21259  total_loss: 0.3748  loss_cls: 0.02256  loss_box_reg: 0.04795  loss_mask: 0.2403  loss_rpn_cls: 0.004891  loss_rpn_loc: 0.008776  time: 0.4425  data_time: 0.0099  lr: 0.001  max_mem: 3702M
[11/30 19:52:29 d2.utils.events]:  eta: 0:27:28  iter: 21279  total_loss: 0.4259  loss_cls: 0.0172  loss_box_reg: 0.03628  loss_mask: 0.257  loss_rpn_cls: 0.00781  loss_rpn_loc: 0.00931  time: 0.4425  data_time: 0.0099  lr: 0.001  max_mem: 3702M
[11/30 19:52:38 d2.utils.events]:  eta: 0:27:19  iter: 21299  total_loss: 0.3417  loss_cls: 0.02753  loss_box_reg: 0.04933  loss_mask: 0.2295  loss_rpn_cls: 0.003625  loss_rpn_loc: 0.005435  time: 0.4426  data_time: 0.0098  lr: 0.001  max_mem: 3702M
[11/30 19:52:47 d2.utils.events]:  eta: 0:27:10  iter: 21319  total_loss: 0.3784  loss_cls: 0.03174  loss_box_reg: 0.05779  loss_mask: 0.2315  loss_rpn_cls: 0.005377  loss_rpn_loc: 0.0114  time: 0.4427  data_time: 0.0102  lr: 0.001  max_mem: 3702M
[11/30

[11/30 19:57:06 d2.utils.events]:  eta: 0:22:58  iter: 21899  total_loss: 0.2925  loss_cls: 0.01728  loss_box_reg: 0.04417  loss_mask: 0.2198  loss_rpn_cls: 0.001428  loss_rpn_loc: 0.004302  time: 0.4436  data_time: 0.0105  lr: 0.001  max_mem: 3702M
[11/30 19:57:15 d2.utils.events]:  eta: 0:22:49  iter: 21919  total_loss: 0.4061  loss_cls: 0.01975  loss_box_reg: 0.04408  loss_mask: 0.2369  loss_rpn_cls: 0.005148  loss_rpn_loc: 0.01339  time: 0.4436  data_time: 0.0101  lr: 0.001  max_mem: 3702M
[11/30 19:57:24 d2.utils.events]:  eta: 0:22:40  iter: 21939  total_loss: 0.429  loss_cls: 0.02804  loss_box_reg: 0.05119  loss_mask: 0.2705  loss_rpn_cls: 0.007568  loss_rpn_loc: 0.008334  time: 0.4436  data_time: 0.0102  lr: 0.001  max_mem: 3702M
[11/30 19:57:33 d2.utils.events]:  eta: 0:22:31  iter: 21959  total_loss: 0.3447  loss_cls: 0.02757  loss_box_reg: 0.05644  loss_mask: 0.2208  loss_rpn_cls: 0.003972  loss_rpn_loc: 0.006897  time: 0.4436  data_time: 0.0099  lr: 0.001  max_mem: 3702M
[1

...

[11/30 20:11:21 d2.utils.events]:  eta: 0:08:42  iter: 23819  total_loss: 0.3463  loss_cls: 0.01996  loss_box_reg: 0.0517  loss_mask: 0.2476  loss_rpn_cls: 0.00378  loss_rpn_loc: 0.005312  time: 0.4439  data_time: 0.0100  lr: 0.001  max_mem: 3839M
[11/30 20:11:29 d2.utils.events]:  eta: 0:08:32  iter: 23839  total_loss: 0.4472  loss_cls: 0.01236  loss_box_reg: 0.04905  loss_mask: 0.3277  loss_rpn_cls: 0.002748  loss_rpn_loc: 0.006487  time: 0.4438  data_time: 0.0095  lr: 0.001  max_mem: 3839M
[11/30 20:11:39 d2.utils.events]:  eta: 0:08:24  iter: 23859  total_loss: 0.4295  loss_cls: 0.03891  loss_box_reg: 0.0798  loss_mask: 0.2775  loss_rpn_cls: 0.005001  loss_rpn_loc: 0.01907  time: 0.4439  data_time: 0.0109  lr: 0.001  max_mem: 3839M
[11/30 20:11:48 d2.utils.events]:  eta: 0:08:15  iter: 23879  total_loss: 0.4034  loss_cls: 0.03099  loss_box_reg: 0.06651  loss_mask: 0.2229  loss_rpn_cls: 0.00668  loss_rpn_loc: 0.018  time: 0.4440  data_time: 0.0101  lr: 0.001  max_mem: 3839M
[11/30 2

[11/30 20:16:05 d2.utils.events]:  eta: 0:03:58  iter: 24459  total_loss: 0.347  loss_cls: 0.02891  loss_box_reg: 0.05126  loss_mask: 0.2519  loss_rpn_cls: 0.004601  loss_rpn_loc: 0.01011  time: 0.4438  data_time: 0.0100  lr: 0.001  max_mem: 3839M
[11/30 20:16:14 d2.utils.events]:  eta: 0:03:50  iter: 24479  total_loss: 0.3605  loss_cls: 0.01979  loss_box_reg: 0.04555  loss_mask: 0.2311  loss_rpn_cls: 0.004277  loss_rpn_loc: 0.00823  time: 0.4438  data_time: 0.0105  lr: 0.001  max_mem: 3839M
[11/30 20:16:23 d2.utils.events]:  eta: 0:03:40  iter: 24499  total_loss: 0.3521  loss_cls: 0.01526  loss_box_reg: 0.03999  loss_mask: 0.222  loss_rpn_cls: 0.002417  loss_rpn_loc: 0.004289  time: 0.4438  data_time: 0.0101  lr: 0.001  max_mem: 3839M
[11/30 20:16:32 d2.utils.events]:  eta: 0:03:32  iter: 24519  total_loss: 0.3343  loss_cls: 0.0242  loss_box_reg: 0.06671  loss_mask: 0.2215  loss_rpn_cls: 0.002356  loss_rpn_loc: 0.007922  time: 0.4438  data_time: 0.0100  lr: 0.001  max_mem: 3839M
[11/3

[11/30 20:20:37 d2.evaluation.evaluator]: Inference done 447/5275. 0.0516 s / img. ETA=0:04:38
[11/30 20:20:42 d2.evaluation.evaluator]: Inference done 536/5275. 0.0515 s / img. ETA=0:04:32
[11/30 20:20:47 d2.evaluation.evaluator]: Inference done 625/5275. 0.0515 s / img. ETA=0:04:26
[11/30 20:20:52 d2.evaluation.evaluator]: Inference done 714/5275. 0.0515 s / img. ETA=0:04:20
[11/30 20:20:57 d2.evaluation.evaluator]: Inference done 801/5275. 0.0516 s / img. ETA=0:04:16
[11/30 20:21:02 d2.evaluation.evaluator]: Inference done 892/5275. 0.0516 s / img. ETA=0:04:10
[11/30 20:21:07 d2.evaluation.evaluator]: Inference done 978/5275. 0.0517 s / img. ETA=0:04:05
[11/30 20:21:12 d2.evaluation.evaluator]: Inference done 1068/5275. 0.0516 s / img. ETA=0:04:00
[11/30 20:21:17 d2.evaluation.evaluator]: Inference done 1159/5275. 0.0516 s / img. ETA=0:03:54
[11/30 20:21:22 d2.evaluation.evaluator]: Inference done 1246/5275. 0.0516 s / img. ETA=0:03:49
[11/30 20:21:27 d2.evaluation.evaluator]: Infer

[11/30 20:25:18 d2.engine.defaults]: Evaluation results for my_dataset_val_v2 in csv format:
[11/30 20:25:18 d2.evaluation.testing]: copypaste: Task: bbox
[11/30 20:25:18 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 20:25:18 d2.evaluation.testing]: copypaste: 44.4138,75.1330,46.8312,30.9838,68.7993,71.6790
[11/30 20:25:18 d2.evaluation.testing]: copypaste: Task: segm
[11/30 20:25:18 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 20:25:18 d2.evaluation.testing]: copypaste: 35.6086,70.8348,33.1467,21.3510,57.2725,64.7646
[11/30 20:25:18 d2.utils.events]:  eta: 0:00:00  iter: 24999  total_loss: 0.3654  loss_cls: 0.02588  loss_box_reg: 0.07489  loss_mask: 0.2474  loss_rpn_cls: 0.004121  loss_rpn_loc: 0.008401  time: 0.4438  data_time: 0.0099  lr: 0.001  max_mem: 3839M
[11/30 20:25:18 d2.engine.hooks]: Overall training speed: 4998 iterations in 0:36:57 (0.4438 s / it)
[11/30 20:25:18 d2.engine.hooks]: Total training time: 0:42:16 (0:05:18 on hooks)


[11/30 20:30:25 d2.evaluation.evaluator]: Total inference time: 0:05:05.027602 (0.057880 s / img per device, on 1 devices)
[11/30 20:30:25 d2.evaluation.evaluator]: Total inference pure compute time: 0:04:34 (0.052056 s / img per device, on 1 devices)
[11/30 20:30:25 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[11/30 20:30:25 d2.evaluation.coco_evaluation]: Saving results to coco_eval/coco_instances_results.json
[11/30 20:30:25 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 0.43 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 0.07 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.444
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.751
 Average Precisio

In [17]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train_v2",) 
cfg.DATASETS.TEST = ("my_dataset_val_v2",)
cfg.TEST.EVAL_PERIOD = 5000
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 4 
cfg.SOLVER.BASE_LR = 0.0001  # pick a good LR
cfg.SOLVER.MAX_ITER = 35000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)
cfg.MAX_SIZE_TRAIN = 756 #Max image size 
cfg.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.OUTPUT_DIR = "./run_equal"
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

In [18]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

[11/30 21:20:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/30 21:20:20 d2.data.datasets.coco]: Loading /application/input/train_annotations_equal.json takes 1.59 seconds.
[11/30 21:20:21 d2.data.datasets.coco]: Loaded 100233 images in COCO format from /application/input/train_annotations_equal.json
[11/30 21:20:27 d2.data.build]: Using training sampler TrainingSampler
[11/30 21:20:27 d2.data.common]: Serializing 100233 elements to byte tensors and concatenating them all ...
[11/30 21:20:27 d2.data.common]: Serialized dataset takes 27.94 MiB
[11/30 21:20:28 d2.engine.train_loop]: Starting training from iteration 30000
[11/30 21:20:38 d2.utils.events]:  eta: 0:39:15  iter: 30019  total_loss: 0.1919  loss_cls: 0.01254  loss_box_reg: 0.01844  loss_mask: 0.1421  loss_rpn_cls: 0.001428  loss_rpn_loc: 0.003821  time: 0.4736  data_time: 0.0279  lr: 0.001  max_mem: 4155M
[11/30 21:20:47 d2.utils.events]:  eta: 0:38:02  iter: 30039  total_loss: 0.2841  loss_cls: 0.011  loss_box_reg: 0.02137  loss_mask: 0.223  loss_rpn_cls: 0.00146  loss_rpn_loc: 0.0

[11/30 21:25:14 d2.utils.events]:  eta: 0:33:17  iter: 30619  total_loss: 0.3274  loss_cls: 0.01329  loss_box_reg: 0.02734  loss_mask: 0.2534  loss_rpn_cls: 0.004751  loss_rpn_loc: 0.007097  time: 0.4596  data_time: 0.0099  lr: 0.001  max_mem: 4355M
[11/30 21:25:23 d2.utils.events]:  eta: 0:33:07  iter: 30639  total_loss: 0.2965  loss_cls: 0.01692  loss_box_reg: 0.02282  loss_mask: 0.2315  loss_rpn_cls: 0.005207  loss_rpn_loc: 0.008472  time: 0.4599  data_time: 0.0100  lr: 0.001  max_mem: 4355M
[11/30 21:25:32 d2.utils.events]:  eta: 0:32:58  iter: 30659  total_loss: 0.2697  loss_cls: 0.01344  loss_box_reg: 0.02365  loss_mask: 0.1991  loss_rpn_cls: 0.002868  loss_rpn_loc: 0.003523  time: 0.4597  data_time: 0.0096  lr: 0.001  max_mem: 4355M
[11/30 21:25:42 d2.utils.events]:  eta: 0:32:50  iter: 30679  total_loss: 0.4341  loss_cls: 0.0186  loss_box_reg: 0.02625  loss_mask: 0.28  loss_rpn_cls: 0.00692  loss_rpn_loc: 0.01054  time: 0.4599  data_time: 0.0097  lr: 0.001  max_mem: 4355M
[11/3

[11/30 21:30:17 d2.utils.events]:  eta: 0:28:57  iter: 31259  total_loss: 0.3721  loss_cls: 0.02212  loss_box_reg: 0.02933  loss_mask: 0.2705  loss_rpn_cls: 0.005528  loss_rpn_loc: 0.01137  time: 0.4663  data_time: 0.0102  lr: 0.001  max_mem: 4355M
[11/30 21:30:27 d2.utils.events]:  eta: 0:28:51  iter: 31279  total_loss: 0.379  loss_cls: 0.02499  loss_box_reg: 0.04396  loss_mask: 0.2586  loss_rpn_cls: 0.003305  loss_rpn_loc: 0.01623  time: 0.4665  data_time: 0.0107  lr: 0.001  max_mem: 4355M
[11/30 21:30:36 d2.utils.events]:  eta: 0:28:42  iter: 31299  total_loss: 0.3095  loss_cls: 0.0147  loss_box_reg: 0.01983  loss_mask: 0.2252  loss_rpn_cls: 0.00325  loss_rpn_loc: 0.008957  time: 0.4666  data_time: 0.0099  lr: 0.001  max_mem: 4355M
[11/30 21:30:45 d2.utils.events]:  eta: 0:28:33  iter: 31319  total_loss: 0.2945  loss_cls: 0.02057  loss_box_reg: 0.02908  loss_mask: 0.213  loss_rpn_cls: 0.003338  loss_rpn_loc: 0.007818  time: 0.4667  data_time: 0.0103  lr: 0.001  max_mem: 4355M
[11/30

[11/30 21:35:19 d2.utils.events]:  eta: 0:24:11  iter: 31899  total_loss: 0.2879  loss_cls: 0.0198  loss_box_reg: 0.03087  loss_mask: 0.192  loss_rpn_cls: 0.003115  loss_rpn_loc: 0.008894  time: 0.4681  data_time: 0.0098  lr: 0.001  max_mem: 4355M
[11/30 21:35:29 d2.utils.events]:  eta: 0:24:03  iter: 31919  total_loss: 0.3221  loss_cls: 0.01399  loss_box_reg: 0.02417  loss_mask: 0.2459  loss_rpn_cls: 0.001739  loss_rpn_loc: 0.006049  time: 0.4682  data_time: 0.0100  lr: 0.001  max_mem: 4355M
[11/30 21:35:38 d2.utils.events]:  eta: 0:23:52  iter: 31939  total_loss: 0.2975  loss_cls: 0.01291  loss_box_reg: 0.01731  loss_mask: 0.2379  loss_rpn_cls: 0.002757  loss_rpn_loc: 0.005997  time: 0.4681  data_time: 0.0102  lr: 0.001  max_mem: 4355M
[11/30 21:35:48 d2.utils.events]:  eta: 0:23:44  iter: 31959  total_loss: 0.4385  loss_cls: 0.01011  loss_box_reg: 0.02193  loss_mask: 0.2751  loss_rpn_cls: 0.002807  loss_rpn_loc: 0.008126  time: 0.4681  data_time: 0.0097  lr: 0.001  max_mem: 4355M
[1

...

[11/30 21:50:49 d2.utils.events]:  eta: 0:09:31  iter: 33819  total_loss: 0.28  loss_cls: 0.01044  loss_box_reg: 0.01936  loss_mask: 0.221  loss_rpn_cls: 0.002783  loss_rpn_loc: 0.007052  time: 0.4758  data_time: 0.0104  lr: 0.001  max_mem: 4355M
[11/30 21:50:59 d2.utils.events]:  eta: 0:09:21  iter: 33839  total_loss: 0.3279  loss_cls: 0.01718  loss_box_reg: 0.03777  loss_mask: 0.2463  loss_rpn_cls: 0.003476  loss_rpn_loc: 0.009653  time: 0.4757  data_time: 0.0107  lr: 0.001  max_mem: 4355M
[11/30 21:51:08 d2.utils.events]:  eta: 0:09:11  iter: 33859  total_loss: 0.3175  loss_cls: 0.01935  loss_box_reg: 0.03239  loss_mask: 0.2196  loss_rpn_cls: 0.003922  loss_rpn_loc: 0.01014  time: 0.4757  data_time: 0.0105  lr: 0.001  max_mem: 4355M
[11/30 21:51:18 d2.utils.events]:  eta: 0:09:01  iter: 33879  total_loss: 0.2708  loss_cls: 0.009941  loss_box_reg: 0.01916  loss_mask: 0.162  loss_rpn_cls: 0.001987  loss_rpn_loc: 0.009911  time: 0.4757  data_time: 0.0105  lr: 0.001  max_mem: 4355M
[11/

[11/30 21:55:51 d2.utils.events]:  eta: 0:04:15  iter: 34459  total_loss: 0.3175  loss_cls: 0.01706  loss_box_reg: 0.02571  loss_mask: 0.1931  loss_rpn_cls: 0.001907  loss_rpn_loc: 0.006387  time: 0.4750  data_time: 0.0100  lr: 0.001  max_mem: 4355M
[11/30 21:56:01 d2.utils.events]:  eta: 0:04:05  iter: 34479  total_loss: 0.2564  loss_cls: 0.01175  loss_box_reg: 0.0212  loss_mask: 0.2013  loss_rpn_cls: 0.001565  loss_rpn_loc: 0.007884  time: 0.4750  data_time: 0.0105  lr: 0.001  max_mem: 4355M
[11/30 21:56:10 d2.utils.events]:  eta: 0:03:55  iter: 34499  total_loss: 0.2667  loss_cls: 0.01574  loss_box_reg: 0.02403  loss_mask: 0.1938  loss_rpn_cls: 0.00599  loss_rpn_loc: 0.01124  time: 0.4750  data_time: 0.0106  lr: 0.001  max_mem: 4355M
[11/30 21:56:20 d2.utils.events]:  eta: 0:03:46  iter: 34519  total_loss: 0.3081  loss_cls: 0.02261  loss_box_reg: 0.0415  loss_mask: 0.1922  loss_rpn_cls: 0.006288  loss_rpn_loc: 0.01149  time: 0.4750  data_time: 0.0102  lr: 0.001  max_mem: 4355M
[11/3

[11/30 22:00:36 d2.evaluation.evaluator]: Inference done 469/5275. 0.0509 s / img. ETA=0:04:23
[11/30 22:00:41 d2.evaluation.evaluator]: Inference done 561/5275. 0.0509 s / img. ETA=0:04:18
[11/30 22:00:46 d2.evaluation.evaluator]: Inference done 653/5275. 0.0510 s / img. ETA=0:04:13
[11/30 22:00:51 d2.evaluation.evaluator]: Inference done 745/5275. 0.0510 s / img. ETA=0:04:08
[11/30 22:00:56 d2.evaluation.evaluator]: Inference done 838/5275. 0.0510 s / img. ETA=0:04:02
[11/30 22:01:01 d2.evaluation.evaluator]: Inference done 930/5275. 0.0510 s / img. ETA=0:03:57
[11/30 22:01:06 d2.evaluation.evaluator]: Inference done 1022/5275. 0.0510 s / img. ETA=0:03:52
[11/30 22:01:11 d2.evaluation.evaluator]: Inference done 1115/5275. 0.0510 s / img. ETA=0:03:47
[11/30 22:01:16 d2.evaluation.evaluator]: Inference done 1206/5275. 0.0511 s / img. ETA=0:03:42
[11/30 22:01:21 d2.evaluation.evaluator]: Inference done 1297/5275. 0.0511 s / img. ETA=0:03:37
[11/30 22:01:26 d2.evaluation.evaluator]: Infe

[11/30 22:05:10 d2.engine.defaults]: Evaluation results for my_dataset_val_v2 in csv format:
[11/30 22:05:10 d2.evaluation.testing]: copypaste: Task: bbox
[11/30 22:05:10 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 22:05:10 d2.evaluation.testing]: copypaste: 46.3376,76.7575,47.9918,32.0903,71.3319,76.3721
[11/30 22:05:10 d2.evaluation.testing]: copypaste: Task: segm
[11/30 22:05:10 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[11/30 22:05:10 d2.evaluation.testing]: copypaste: 37.7392,72.8567,35.8773,23.2267,60.1363,69.0809
[11/30 22:05:10 d2.utils.events]:  eta: 0:00:00  iter: 34999  total_loss: 0.2514  loss_cls: 0.0138  loss_box_reg: 0.03072  loss_mask: 0.1831  loss_rpn_cls: 0.002404  loss_rpn_loc: 0.006418  time: 0.4740  data_time: 0.0101  lr: 0.001  max_mem: 4355M
[11/30 22:05:10 d2.engine.hooks]: Overall training speed: 4998 iterations in 0:39:29 (0.4740 s / it)
[11/30 22:05:10 d2.engine.hooks]: Total training time: 0:44:40 (0:05:11 on hooks)
[

[11/30 22:10:11 d2.evaluation.evaluator]: Total inference pure compute time: 0:04:39 (0.053015 s / img per device, on 1 devices)
[11/30 22:10:11 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[11/30 22:10:11 d2.evaluation.coco_evaluation]: Saving results to coco_eval/coco_instances_results.json
[11/30 22:10:11 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 0.37 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 0.04 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.463
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.768
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | ma

In [35]:
#Create submission. 
!python module_submittion.py --model_path="run_equal/" #overload: https://github.com/pytorch/vision/pull/2705

1.6.0 True
Start creating files
input/test_v2/
/opt/conda/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:124: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629403081/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  filter_inds = filter_mask.nonzero()
0 15606
1000 15606
2000 15606
3000 15606
4000 15606
5000 15606
6000 15606
7000 15606
8000 15606
9000 15606
10000 15606
11000 15606
12000 15606
13000 15606
14000 15606
15000 15606
0 15606
1000 15606
2000 15606
3000 15606
4000 15606
5000 15606
6000 15606
7000 15606
8000 15606
9000 15606
10000 15606
11000 15606
12000 15606
13000 15606
14000 15606
15000 15606
Detectron2:  3060 instances,  2296 images
Detectron2:  2661 instances,  2075 images
Done!
